## Content
1. [Introduction](#1.-Introduction)<br>
    1.1. [Author's Details](#1.1.-Author's-Details)<br>
    1.2. [Downloading and Importing the required Libraries](#1.2.-Downloading-and-Importing-the-required-Libraries)<br>
    1.3. [Loading the Dataset](#1.3.-Loading-the-Dataset)<br>
    1.4. [Splitting the Dataset into Train and Test](#1.4.-Splitting-the-Dataset-into-Train-and-Test)<br>
    1.5. [Describing the parameters for the model](#1.5.-Describing-the-parameters-for-the-model)<br>
2. [Tokenization](#2-Tokenization)<br>
    2.1. [Importing the Tokenizer](#2.1.-Importing-the-Tokenizer)<br>
    2.2. [Adding the summarization prefix](#2.2.-Adding-the-summarization-prefix)<br>
3. [Preprocessing](#3.-Preprocessing)<br>
    3.1. [Defining the Preprocessing Function](#3.1.-Defining-the-Preprocessing-Function)<br>
    3.2. [Mapping the Preprocessing Function to the Train and Test Datasets](#3.2.-Mapping-the-Preprocessing-Function-to-the-Train-and-Test-Datasets)<br>
4. [Training](#4.-Training)<br>
    4.1. [Improting the Tensorflow Model](#4.1.-Improting-the-Tensorflow-Model)<br>
    4.2. [Using the DataCollatorForSeq2Seq for collating the data](#4.2.-Using-the-DataCollatorForSeq2Seq-for-collating-the-data)<br>
    4.3. [Creating the Training, Test and Generation Datasets](#4.3.-Creating-the-Training,-Test-and-Generation-Datasets)<br>
5. [Fine-Tuning](#5.-Fine-Tuning)<br>
    5.1. [Tuning optimizer and compiling the model](#5.1.-Tuning-optimizer-and-compiling-the-model)<br>
    5.2. [Model Summary](#5.2.-Model-Summary)<br>
6. [Evaluation](#6.-Evaluation)<br>
    6.1. [Importing the RougeL Metric](#6.1.-Importing-the-RougeL-Metric)<br>
7. [Training the Model](#7.-Training-the-Model)<br>
    7.1. [Training the Model](#7.1.-Training-the-Model)<br>
8. [Testing the Model](#8.-Testing-the-Model)<br>
    8.1. [Testing the Model](#8.1.-Testing-the-Model)<br>
9. [Saving the Model](#9.-Saving-the-Model)<br>

# 1. Introduction

# 1.1. Author's Details

## Name : Nisarg Patel <br>
## PRN : 21070126060 <br>
## Batch : AIML - A3 <br>
## Git-Repo : [GitRepo](https://github.com/SnakeEyes1308/NLP) <br>

# 1.2. Downloading and Importing the required Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers -q
!pip install keras_nlp -q
!pip install datasets -q
!pip install huggingface-hub -q
!pip install rouge-score -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Importing the needed libraries
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

D0927 13:09:51.240667260      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0927 13:09:51.240691774      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0927 13:09:51.240695346      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0927 13:09:51.240698246      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0927 13:09:51.240700761      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0927 13:09:51.240703686      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0927 13:09:51.240706300      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0927 13:09:51.

# 1.3. Loading the Dataset

In [3]:
# Loading the Xsum Dataset
from datasets import load_dataset
dataset = load_dataset("xsum", split="train")
print(dataset)

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 11334/11334 [00:31<00:00, 355.24 examples/s]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})


In [4]:
# Printing the first element of the dataset
print(dataset[0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

# 1.4. Splitting the Dataset into Train and Test

In [5]:
# Splitting the dataset into train and test
datasets = dataset.train_test_split(train_size=0.1,test_size=0.1)

In [6]:
# Checking the length of the train and test split
print(len(datasets['train']))
print(len(datasets['test']))

20404
20405


# 1.5. Describing the parameters for the model

In [7]:
# Define maximum input length for tokenization
MAX_INPUT_LENGTH = 1024

# Define minimum and maximum target lengths for summarization
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128

# Define batch size for training
BATCH_SIZE = 8

# Define learning rate for optimizer
LEARNING_RATE = 2e-5

# Define maximum number of epochs for training
MAX_EPOCHS = 1

# Define the pre-trained T5 model checkpoint to be used
MODEL_CHECKPOINT = "t5-small" # Name of Model


# 2. Tokenization

# 2.1. Importing the Tokenizer

In [8]:
# Importing the tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# 2.2. Adding the summarization prefix

In [9]:
# Check if the specified model checkpoint is either "t5-small" or "t5-base"
if MODEL_CHECKPOINT in ["t5-small", "t5-base"]:
    # If yes, set the prefix for summarization task
    prefix = "summarize: "
else:
    # If not, set an empty prefix
    prefix = ""


# 3. Preprocessing

# 3.1. Defining the Preprocessing Function

In [10]:
#Preprocessing
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH,truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 3.2. Mapping the Preprocessing Function to the Train and Test Datasets

In [11]:
# Mapping the preprocessing function to the train and test datasets
tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/20404 [00:00<?, ? examples/s]/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 20405/20405 [00:12<00:00, 1615.06 examples/s]


# 4. Training

# 4.1. Improting the Tensorflow Model

In [12]:
# Using tensorflow AutoModelForSeq2SeqLM for summarization
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# 4.2. Using the DataCollatorForSeq2Seq for collating the data

In [13]:
# Using the DataCollatorForSeq2Seq for collating the data
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model,return_tensors="tf")

# 4.3. Creating the Training, Test and Generation Datasets

In [14]:
# Create a training dataset from the tokenized training data
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,                 # Set the batch size for training data
    columns=["input_ids", "attention_mask", "labels"],  # Specify columns to include in the dataset
    shuffle=True,                          # Shuffle the data for randomness
    collate_fn=data_collator               # Use the specified data collator function
)

# Create a test dataset from the tokenized test data
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,                 # Set the batch size for test data
    columns=["input_ids", "attention_mask", "labels"],  # Specify columns to include in the dataset
    shuffle=False,                         # Do not shuffle the test data
    collate_fn=data_collator               # Use the specified data collator function
)

# Create a generation dataset by selecting a subset of data from the tokenized test data
generation_dataset = (
    tokenized_datasets["test"]             # Access the tokenized test data
    .shuffle()                             # Shuffle the data for randomness
    .select(list(range(200)))              # Select a specific range of samples
    .to_tf_dataset(                        # Convert the selected data to a TF dataset
        batch_size=BATCH_SIZE,             # Set the batch size
        columns=["input_ids", "attention_mask", "labels"],  # Specify columns to include
        shuffle=False,                     # Do not shuffle the generation data
        collate_fn=data_collator           # Use the specified data collator function
    )
)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# 5. Fine-Tuning

# 5.1. Tuning optimizer and compiling the model

In [15]:
# Tuning the optimizer and compiling the model
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

# 5.2. Model Summary

In [16]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60,506,624
Trainable params: 60,506,624
Non-trainable params: 0
_________________________________________________________________


# 6. Evaluation

# 6.1. Importing the RougeL Metric

In [17]:
# Import the RougeL metric from keras_nlp.metrics module
import keras_nlp
rouge_l = keras_nlp.metrics.RougeL()

# Define a function to compute the RougeL metric
def metric_fn(eval_predictions):
    # Unpack the predictions and labels
    predictions, labels = eval_predictions

    # Decode the predicted tokens to text
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace masked label tokens
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id

    # Decode the label tokens to text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute the RougeL score
    result = rouge_l(decoded_labels, decoded_predictions)

    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result


Using TensorFlow backend


# 7. Training the Model

# 7.1. Training the Model

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn,eval_dataset=generation_dataset, predict_with_generate=True)
callbacks = [metric_callback]
# For now we will use our test set as our validation_data
model.fit(train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS,callbacks=callbacks)

https://symbolize.stripped_domain/r/?trace=7af9b354c4f9,7af9b3486f8f,7af842acb0e8,7af842ac8692,7af842ac8b96,7af84387d09a,7af8333f5a7c,7af82d829614,7af82d82a1ea,7af8333ed5fb,7af82d823975,7af82d8265d3,7af82d80b148,7af8333fe243,7af8393b835a,7af84214ae24,7af71777ca22,7af71777cd13,7af717780615,7af9b377fb3d,100000000&map=aef7fe2e538f701f46d88df9ee3b51d79ec62b1e:7af84227b000-7af844256728,70f32832c9000a2f02b2bd353a9dfb35c743ca88:7af842017000-7af842264750,8f79f803f683427be94b1cfeea32716e6ef365e4:7af829915000-7af84186a830
*** SIGTERM received by PID 15 (TID 15) on cpu 44 from PID 1; stack trace: ***
PC: @     0x7af9b354c4f9  (unknown)  syscall
    @     0x7af828c3605a       1152  (unknown)
    @     0x7af9b3486f90  1872873984  (unknown)
    @     0x7af842acb0e9        176  nsync::nsync_sem_wait_with_cancel_()
    @     0x7af842ac8693        144  nsync::nsync_cv_wait_with_deadline_generic()
    @     0x7af842ac8b97         32  nsync::nsync_cv_wait_with_deadline()
    @     0x7af84387d09b        464  tensorflow::ProcessFunctionLibraryRuntime::RunSync()
    @     0x7af8333f5a7d        368  tensorflow::KernelAndDeviceFunc::Run()
    @     0x7af82d829615        480  tensorflow::EagerKernelExecute()
    @     0x7af82d82a1eb        528  tensorflow::ExecuteNode::Run()
    @     0x7af8333ed5fc         96  tensorflow::EagerExecutor::SyncExecute()
    @     0x7af82d823976       1152  tensorflow::(anonymous namespace)::EagerLocalExecute()
    @     0x7af82d8265d4       1136  tensorflow::EagerExecute()
    @     0x7af82d80b149        208  tensorflow::EagerOperation::Execute()
    @     0x7af8333fe244        208  tensorflow::CustomDeviceOpHandler::Execute()
    @     0x7af8393b835b         80  TFE_Execute
    @     0x7af84214ae25       1440  TFE_Py_ExecuteCancelable()
    @     0x7af71777ca23        208  tensorflow::TFE_Py_ExecuteCancelable_wrapper()
    @     0x7af71777cd14        208  pybind11::cpp_function::initialize<>()::{lambda()#3}::_FUN()
    @     0x7af717780616        768  pybind11::cpp_function::dispatcher()
    @     0x7af9b377fb3e  (unknown)  cfunction_call_varargs
    @        0x100000001  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7af9b354c4f9,7af828c36059,7af9b3486f8f,7af842acb0e8,7af842ac8692,7af842ac8b96,7af84387d09a,7af8333f5a7c,7af82d829614,7af82d82a1ea,7af8333ed5fb,7af82d823975,7af82d8265d3,7af82d80b148,7af8333fe243,7af8393b835a,7af84214ae24,7af71777ca22,7af71777cd13,7af717780615,7af9b377fb3d,100000000&map=aef7fe2e538f701f46d88df9ee3b51d79ec62b1e:7af84227b000-7af844256728,70f32832c9000a2f02b2bd353a9dfb35c743ca88:7af842017000-7af842264750,8f79f803f683427be94b1cfeea32716e6ef365e4:7af829915000-7af84186a830,1278088d049ad36cb636fbbc76303cb3:7af81d600000-7af828e4d7c0 
E0927 15:10:54.481297      15 coredump_hook.cc:360] RAW: Remote crash gathering disabled for SIGTERM.

#### Sir i have tried using TPU's GPU's and normal CPU's for the computation purpose but since the dataset is too large everytime it was either thorwing oom error or it used to get disconnected. Despite of trying for 2 days i was unable to train the model however upon reaching the college i will use the GPU lab and get this model trained over there meanwhile i will be trying to train the model on google colab and kaggle as well if it get's trained i will update the file in the github where this is uploaded. Thanks sir for understanding.

# 8. Testing the Model

# 8.1. Testing the Model

In [ ]:
# Import the summarization pipeline from transformers library
from transformers import pipeline

# Create a summarization pipeline with the specified model and tokenizer
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

# Print a separator line
print("-" * 1000)

# Print the original article content
print("________Original Article__________")
print(datasets['test'][0]['document'])

# Print a separator line
print("-" * 1000)

# Print the original summary
print("________Original Summary__________")
print(datasets['test'][0]['summary'])

# Print a separator line
print("-" * 1000)

# Generate and print the predicted summary
summarizer(datasets["test"][0]["document"], min_length=MIN_TARGET_LENGTH, max_length=MAX_TARGET_LENGTH)


# 9. Saving the Model

In [ ]:
# Saving the model
model.save_pretrained("T5CustomSummarizer",from_pt=True)
tokenizer.save_pretrained("T5CustomSumTokenizer",from_pt=True)